# Multiple Objects

A real analysis rarely deals with single objects - just one electron or one jet. More common are combinations of objects. The `hep_tables` package does this by leaning on the `map` function previously described and python's captured variables.

In [1]:
import sys
sys.path = ['..'] + sys.path
from hep_tables import xaod_table, make_local
from dataframe_expressions import user_func
from func_adl import EventDataset
import matplotlib.pyplot as plt

import numpy as np

plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

# Define DeltaR function availible on servicex (see the functions intro for more info)
@user_func
def DeltaR(p1_eta: float, p1_phi: float, p2_eta: float, p2_phi: float) -> float:
    '''
    Calculate the DeltaR between two particles given their `eta` and `phi` locations.
    Implemented on the back end.
    '''
    assert False, 'This should never be called'
    
define_alias('', 'ptgev', lambda o: o.pt / 1000.0)

In [2]:
dataset = EventDataset('localds://mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00')
df = xaod_table(dataset)

## Matching the MC electron to the actual electron

We need to look at the MC particles, select the electrons, and then match them to electrons in the actual dataset. I've not done this in this dataset before, so I'm going to have to feel my way a bit with a few plots to allow me to understand what is going on.

First, lets get a plot of the MC electrons. I had to look up the pdg id (embarassing): `11` - Note that -11 isn't being used here because of a bug that this turned up in the backend (see [bug](https://github.com/iris-hep/func_adl_xAOD/issues/37))!

In [3]:
mc_part = df.TruthParticles('TruthParticles')
mc_ele = mc_part[(mc_part.pdgId == 11)]

In [25]:
%%time
mc_ele_pt = make_local(mc_ele.pt)
mc_ele_eta = make_local(mc_ele.eta)
mc_ele_phi = make_local(mc_ele.phi)

Wall time: 9min 23s


In [ ]:
%%time
eles = df.Electrons('Electrons')
ele_pt = make_local(eles.pt)
ele_eta = make_local(eles.eta)
ele_phi = make_local(eles.phi)

In [ ]:
plt.hist(mc_ele_pt.flatten(), label='mc', bins=50, range=(0.0, 200.0), density=True, histtype='step')
plt.hist(ele_pt.flatten(), label='reco', bins=50, range=(0.0, 200.0), density=True, histtype='step')
plt.legend()
plt.xlabel('Electron $p_T$')
_ = plt.ylabel('Noramlized Counts')

In [ ]:
plt.hist(mc_ele_eta.flatten(), label='mc', bins=50, range=(-3.0, 3.0), density=True, histtype='step')
plt.hist(ele_eta.flatten(), label='reco', bins=50, range=(-3.0, 3.0), density=True, histtype='step')
plt.legend()
plt.xlabel('Electron $\eta$')
_ = plt.ylabel('Noramlized Counts')

In [ ]:
plt.hist(mc_ele_phi.flatten(), label='mc', bins=50, range=(0.0, 3.2), density=True, histtype='step')
plt.hist(ele_phi.flatten(), label='reco', bins=50, range=(0.0, 3.2), density=True, histtype='step')
plt.legend()
plt.xlabel('Electron $\phi$')
_ = plt.ylabel('Noramlized Counts')